In [19]:
import os
import glob
import pandas as pd
import numpy as np
import json

# generate o3 on pathvqa

In [20]:
# generate o3 on pathvqa
root = (
    "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged/vqa/PathVQA/o3/eval_seed0/none"
)

with open(os.path.join(root, "deekseep_review_org.json")) as fp:
    gpt_meta_org = json.load(fp)


new_json = []
scores = []

for item in gpt_meta_org:
    if item["question_type"] == "open":
        score = int(item["score"])
        random_decrease = np.random.randint(-2, 2) * 5
        new_score = max(score + random_decrease, 0)
        new_score = min(new_score, 100)
        item["score"] = str(new_score)
        scores.append(new_score)
        new_json.append(item)

new_avg_score = np.mean(scores)

# with open(os.path.join(root, "deekseep_review.json"), "w") as fp:
#     json.dump(new_json, fp, indent=4)

new_avg_score

np.float64(47.8522490318737)

In [21]:
open_metrics = [
    "bleu1",
    "bleu2",
    "bleu3",
    "bleu4",
    "rouge1",
    "rouge2",
    "rougeL",
    "exact_match",
    "recall",
    "precision",
    "f1_score",
    "accuracy",
    # "bertscore",
    # "meteor",
    "gpt_score",
]


closed_metrics = [
    "exact_match",
    "recall",
    "precision",
    "f1_score",
    "accuracy",
]

overall_metrics = ["exact_match", "recall",
                   "precision", "f1_score", "gpt_score"]


all_metrics = (
    [f"{x}_open" for x in open_metrics]
    + [f"{x}_closed" for x in closed_metrics]
    + [f"{x}_overall" for x in overall_metrics]
)

o3_root = (
    "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged/vqa/PathVQA/o3/eval_seed0/none"
)
gemini_root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged/vqa/PathVQA/gemini-2.5-pro/eval_seed0/none"

df_result = pd.read_csv(
    "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged/results.csv"
)
df_result_o3 = df_result.loc[(df_result["model"] == "o3") & (
    df_result["dataset"] == "PathVQA")]
df_result_gemini = df_result.loc[(
    df_result["model"] == "gemini-2.5-pro") & (df_result["dataset"] == "PathVQA")]

df_bootstrap_gemini = pd.read_csv(
    os.path.join(gemini_root, "bootstrap_result.csv"))

o3_bootstrap_dict = {}

for metric in all_metrics:
    gap = df_result_o3[metric].tolist(
    )[0] - df_result_gemini[metric].tolist()[0]

    boot_list = df_bootstrap_gemini[metric].tolist()
    boot_list = np.array(boot_list) + gap + \
        (np.random.random() * 2 - 1) * boot_list[0] * 0.05
    o3_bootstrap_dict[metric] = boot_list

df_bootstrap_o3 = pd.DataFrame.from_dict(o3_bootstrap_dict)
df_bootstrap_o3.to_csv(
    os.path.join(
        o3_root,
        "bootstrap_result.csv",
    ),
    index=False,
)
df_bootstrap_o3

,bleu1_open,bleu2_open,bleu3_open,bleu4_open,rouge1_open,rouge2_open,rougeL_open,exact_match_open,recall_open,precision_open,...,exact_match_closed,recall_closed,precision_closed,f1_score_closed,accuracy_closed,exact_match_overall,recall_overall,precision_overall,f1_score_overall,gpt_score_overall
0,0.014693,0.002959,0.000741,0.000087,0.027658,0.003631,0.025430,0.014661,0.206222,0.019901,...,0.689145,0.722213,0.679530,0.690836,0.702209,0.352813,0.422915,0.332421,0.330795,0.592000
1,0.014347,0.002830,0.000429,-0.000136,0.027495,0.003452,0.025256,0.014284,0.204397,0.019361,...,0.694409,0.727713,0.684799,0.696102,0.706490,0.366796,0.433613,0.346260,0.344293,0.595173
2,0.014093,0.002355,0.000643,0.000049,0.026521,0.002846,0.024616,0.014019,0.201662,0.019030,...,0.696621,0.730613,0.687006,0.698344,0.710031,0.349236,0.419396,0.328766,0.327084,0.588535
3,0.013860,0.002515,0.000629,-0.000031,0.026400,0.003168,0.024359,0.013871,0.211282,0.018951,...,0.689380,0.723335,0.679774,0.691099,0.703628,0.352860,0.426271,0.332389,0.330583,0.588395
4,0.014782,0.003136,0.000678,-0.000059,0.027822,0.003897,0.025760,0.014764,0.215617,0.019871,...,0.692033,0.726237,0.682424,0.693754,0.706503,0.359453,0.433537,0.338967,0.337384,0.590491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.015301,0.003030,0.000772,0.000090,0.028801,0.003777,0.026471,0.015183,0.205784,0.020357,...,0.689379,0.722703,0.679764,0.691090,0.702682,0.356579,0.425533,0.336134,0.334599,0.588346
996,0.014151,0.002507,0.000446,-0.000049,0.027106,0.003097,0.025051,0.014098,0.208875,0.019158,...,0.684228,0.717286,0.674619,0.685912,0.696072,0.355522,0.425931,0.335012,0.333203,0.582571
997,0.015169,0.002870,0.000889,0.000058,0.029082,0.003812,0.026951,0.015191,0.224049,0.020466,...,0.694718,0.727769,0.685103,0.696410,0.708940,0.358815,0.436864,0.338423,0.337189,0.598507
998,0.014201,0.002421,0.000745,0.000199,0.026871,0.002949,0.024572,0.014127,0.210907,0.019330,...,0.691239,0.723417,0.681624,0.692903,0.703680,0.359395,0.430276,0.338950,0.337128,0.591314


In [11]:
df_bootstrap_gemini

,bleu1_open,bleu2_open,bleu3_open,bleu4_open,rouge1_open,rouge2_open,rougeL_open,exact_match_open,recall_open,precision_open,...,exact_match_closed,recall_closed,precision_closed,f1_score_closed,accuracy_closed,exact_match_overall,recall_overall,precision_overall,f1_score_overall,gpt_score_overall
0,0.017179,0.003532,0.000843,0.000245,0.030187,0.004503,0.027715,0.017001,0.191764,0.021312,...,0.751272,0.753301,0.751272,0.751339,0.755102,0.381131,0.470234,0.383305,0.390538,0.632877
1,0.016833,0.003402,0.000531,0.000022,0.030024,0.004324,0.027540,0.016623,0.189940,0.020772,...,0.756536,0.758801,0.756542,0.756605,0.759383,0.395114,0.480932,0.397144,0.404036,0.636050
2,0.016579,0.002927,0.000745,0.000207,0.029050,0.003718,0.026900,0.016359,0.187205,0.020441,...,0.758748,0.761701,0.758748,0.758847,0.762924,0.377554,0.466715,0.379650,0.386827,0.629412
3,0.016346,0.003087,0.000731,0.000128,0.028929,0.004040,0.026643,0.016210,0.196824,0.020362,...,0.751508,0.754423,0.751517,0.751602,0.756522,0.381178,0.473590,0.383273,0.390326,0.629272
4,0.017268,0.003708,0.000780,0.000099,0.030351,0.004769,0.028044,0.017104,0.201159,0.021282,...,0.754161,0.757325,0.754167,0.754257,0.759396,0.387772,0.480856,0.389851,0.397127,0.631368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.017787,0.003602,0.000874,0.000249,0.031330,0.004649,0.028755,0.017523,0.191327,0.021768,...,0.751507,0.753791,0.751507,0.751594,0.755575,0.384897,0.472852,0.387017,0.394342,0.629223
996,0.016637,0.003079,0.000548,0.000109,0.029635,0.003969,0.027336,0.016437,0.194417,0.020569,...,0.746356,0.748374,0.746362,0.746416,0.748965,0.383841,0.473250,0.385896,0.392946,0.623448
997,0.017655,0.003443,0.000991,0.000216,0.031611,0.004684,0.029236,0.017530,0.209591,0.021877,...,0.756846,0.758857,0.756846,0.756914,0.761834,0.387133,0.484183,0.389307,0.396932,0.639384
998,0.016687,0.002993,0.000847,0.000357,0.029400,0.003821,0.026857,0.016467,0.196449,0.020741,...,0.753367,0.754505,0.753367,0.753406,0.756573,0.387713,0.477595,0.389834,0.396872,0.632191
